In [ ]:
import re

def limpiar_articulo(texto):
    # Eliminar la sección de resumen (abstract)
    texto = re.sub(r'(?i)resumen\s*:.*?(?=\n\n|\n\s*\w)', '', texto, flags=re.DOTALL)
    texto = re.sub(r'(?i)abstract\s*:.*?(?=\n\n|\n\s*\w)', '', texto, flags=re.DOTALL)

    # Eliminar palabras clave (keywords)
    texto = re.sub(r'(?i)palabras clave\s*:.*?(?=\n\n|\n\s*\w)', '', texto, flags=re.DOTALL)
    texto = re.sub(r'(?i)keywords\s*:.*?(?=\n\n|\n\s*\w)', '', texto, flags=re.DOTALL)

    return texto.strip()

# Ejemplo de texto
articulo = """Título: Inteligencia Artificial
Resumen: Este artículo explora el impacto de la IA en la sociedad.
Palabras clave: IA, Machine Learning, Automatización
Introducción: La inteligencia artificial ha avanzado significativamente..."""

# Limpiar el artículo
texto_limpio = limpiar_articulo(articulo)
print(texto_limpio)

Título: Inteligencia Artificial


Introducción: La inteligencia artificial ha avanzado significativamente...


In [187]:
import os
import re
from pdfminer.high_level import extract_pages
from pdfminer.high_level import LTPage
from pdfminer.layout import LTTextContainer, LTTextLineVertical, LTTextBoxHorizontal, LTTextBox, Rect

In [157]:
os.makedirs("data", exist_ok=True)
articles = [os.path.join("data", art) for art in os.listdir("data")]
articles

['data\\000.pdf',
 'data\\001.pdf',
 'data\\002.pdf',
 'data\\003.pdf',
 'data\\004.pdf',
 'data\\101404-Texto del artículo-379745-1-10-20230814.pdf',
 'data\\103822-Texto del artículo-398336-1-10-20240109.pdf',
 'data\\103827-Texto del artículo-398338-1-10-20240109.pdf',
 'data\\108896-Texto del artículo-430136-1-10-20240723.pdf',
 'data\\110685-Texto del artículo-444650-1-10-20241017.pdf',
 'data\\113135-Texto del artículo-461992-1-10-20250121.pdf',
 'data\\130-Texto del artículo en fichero de Microsoft Word o LibreOffice (necesario)-134-1-10-20090331.pdf',
 'data\\32502-Texto del artículo-103690-1-10-20150102 (1).pdf',
 'data\\32502-Texto del artículo-103690-1-10-20150102.pdf',
 'data\\32511-Texto del artículo-103699-1-10-20150102.pdf',
 'data\\32519-Texto del artículo-103707-1-10-20150102.pdf',
 'data\\32576-Texto del artículo-103765-1-10-20150102.pdf',
 'data\\32585-Texto del artículo-103775-1-10-20150102.pdf',
 'data\\32738-Texto del artículo-103935-1-10-20150102.pdf',
 'data\\32

**arreglar**:

etno-\ngráfico -> etnográfico -> .replace("-\n", "")

**tomar en cuenta toda la pagina hasta donde aparece el primer patron de pie de pagina**

**revisar**
- los articulos viejos de aibr tienen una linea que separa el header y el contenido principal

**mm**:
- los headers se repiten al menos (len - 2) / 2 veces en el texto

In [170]:
import layoutparser as lp

pdf_layout = lp.load_pdf(articles[3])
page = pdf_layout[5]
for block in page:
	print(block)



TextBlock(block=Rectangle(x_1=206.6034, y_1=32.76580000000013, x_2=265.8234, y_2=39.76580000000013), text=González-Caniulef,, id=0, type=VNGVXH+HelveticaNeueLTPro-Roman, parent=None, next=None, score=None)
TextBlock(block=Rectangle(x_1=267.7694, y_1=32.76580000000013, x_2=273.99240000000003, y_2=39.76580000000013), text=E., id=1, type=VNGVXH+HelveticaNeueLTPro-Roman, parent=None, next=None, score=None)
TextBlock(block=Rectangle(x_1=275.9384, y_1=32.76580000000013, x_2=283.1974, y_2=39.76580000000013), text=G., id=2, type=VNGVXH+HelveticaNeueLTPro-Roman, parent=None, next=None, score=None)
TextBlock(block=Rectangle(x_1=285.14340000000004, y_1=32.76580000000013, x_2=299.01040000000006, y_2=39.76580000000013), text=Rev., id=3, type=LQINNB+HelveticaNeueLTPro-It, parent=None, next=None, score=None)
TextBlock(block=Rectangle(x_1=300.95640000000003, y_1=32.76580000000013, x_2=314.6974000000001, y_2=39.76580000000013), text=Esp., id=4, type=LQINNB+HelveticaNeueLTPro-It, parent=None, next=None,

In [162]:
def extract_page(pdf_path, page):
    pages = extract_pages(pdf_path)
    for i, page_layout in enumerate(pages):
        if i != page - 1: continue
        return page_layout
    return None

In [217]:
# regex stop conditions
references_re = r"(?i)^\s*(referencias|bibliografía citada|bibliografia citada|bibliografía|bibliografia|citas|fuentes|referencias bibliograficas|referencias bibliográficas)\s*$"

# regex delete
version_logs = r"(?i)(recibido|aceptado|publicado|(segunda|tercera|cuarta|quinta|sexta) versión)[,:]?\s*(el\s*)?(\d{1,2}(\s*de)?\s*[a-záéíóú]+(\s*de\s*\d{4})?|\d{1,2}/\d{1,2}/\d{4})(\s*\.\s*)?"
footnote_patterns = r"(?m)^(?:\d+|[*†‡¹²³⁴⁵⁶⁷⁸⁹]+|\[\d+\])\s+.*$"
"""                                                                        ^
                                                                           |
                                       								  punto opcional
"""
footnote_references = r"\.[0-9]+|[0-9]+\."

In [218]:
def extract_possible_header(pdf_path: str) -> set[str]:
    return extract_possible_section(pdf_path, "header")

def extract_possible_footer(pdf_path: str) -> set[str]:
    return extract_possible_section(pdf_path, "footer")

def extract_possible_section(pdf_path: str, section: str) -> set[str]:
    counter: map[str, int] = {}
    bbox_index = {
        "footer": lambda bbox, height, cutoff: bbox[3] <= cutoff,
        "header": lambda bbox, height, cutoff: bbox[1] >= height - cutoff,
    }
    func_belonging = bbox_index.get(section)
    if func_belonging is None:
        raise ValueError(f"Invalid section: {section}")

    for page in extract_pages(pdf_path):
        height = page.bbox[3]
        cutoff = height * 0.15

        for element in page:
            if not isinstance(element, (LTTextBox, LTTextBox, LTTextContainer)):
                continue
            if not func_belonging(element.bbox, height, cutoff):
                continue
            txt = element.get_text().strip()
            counter[txt] = counter.get(txt, 0) + 1
    
    return {k for k, v in counter.items() if v > 1 and len(k) > 0}

**los footnote references se llevan los años, en textos como: en 1592. Aquello [...]**

In [228]:
def check_stop_conditions(text) -> bool:
	return any([
		re.search(references_re, text, flags=re.MULTILINE),
	])

def clean_unwanted_patterns(text) -> str:
	# Eliminar patrones como "Recibido el 12 de marzo de 2021."
	text = re.sub(version_logs, "", text).strip()
	
	# text = re.sub(footnote_patterns, "", text, flags=re.MULTILINE).strip()

	# Eliminar las referencias de las notas al pie y reemplazarlas por un punto
	# text = re.sub(footnote_references, ".", text).strip()

	return text

def avoid_unwanted_texts(text) -> str:
	return text

def clean_page(text) -> str:
	text = text.replace("-\n", "")
	return text

def extract_page_text(page, headers: set[str], footers: set[str]) -> str:
	text = ""

	for element in page:
		if not isinstance(element, (LTTextBoxHorizontal, LTTextBox, LTTextContainer)):
			continue
		txt = element.get_text().strip()
		if txt.isdigit() or txt in headers or txt in footers:
			continue

		txt = avoid_unwanted_texts(txt)
		if check_stop_conditions(txt):
			break

		text += txt

	text = clean_unwanted_patterns(text)
	text = clean_page(text)
	return text

In [229]:
art = articles[1]
headers = extract_possible_header(art)
footers = extract_possible_footer(art)
page = extract_page(art, 6)
print(extract_page_text(page, headers, footers))

En efecto, la documentación notarial limeña indica que Belveder llegó al Perú a hacer fortuna mediante la actividad comercial. El registro más
antiguo que se ha encontrado es un poder otorgado en 1584 por el regidor
de Lima, don Luis Rodríguez de la Serna, quien le brindaba poder general para cobrar plata a sus deudores, entregar cartas de pago y realizar finiquitos.16 En 1589 se le encuentra recibiendo poder de Juan Sánchez, alguacil  de  corte,  para  que  Belveder  —quien  se  hallaba  de  partida  a  Tierra
Firme— cobrase de Alonso Ramos más de 700 pesos que le había entregado para emplear mercaderías.17 Desde entonces el aragonés no aparece
efectuando  transacciones,  probablemente  por  hallarse  fuera  de  la  capital,
hasta  1592.  En  enero  de  este  año  Belveder  concedió  poder  a  su  primo
Cristóbal de Tudela, residente en Chile, para que cobrase a sus deudores,
especialmente a Luis[?] de Agurto y Pedro de Ormachea, quienes le debían  cierta  cantidad  de  pesos.18 Casi 

In [194]:
page = extract_page(articles[0], 5)
for element in page:
    print(element)

<LTTextBoxHorizontal(0) 92.700,748.933,106.756,757.401 '462\n'>
<LTTextBoxHorizontal(1) 271.980,747.891,376.029,757.318 'Segundas generaciones \n'>
<LTTextBoxHorizontal(2) 92.700,712.398,522.450,723.688 'ha  cuajado  en  fórmulas  distintas  y  más  o  menos  eficaces  en  cada  uno  de  esos \n'>
<LTTextBoxHorizontal(3) 92.700,692.898,522.335,704.188 'estados que, de todos modos, han mantenido cierta heterogeneidad. En tercer lugar, \n'>
<LTTextBoxHorizontal(4) 92.700,673.458,522.449,684.748 'en  los  años  de  posguerra  y  hasta  los  setenta,  se  desarrollaron  modelos  de \n'>
<LTTextBoxHorizontal(5) 92.700,653.958,522.414,665.248 'incorporación temporal de población inmigrante gracias a una elevada demanda de \n'>
<LTTextBoxHorizontal(6) 92.700,634.519,522.475,645.808 'mano  de  obra  que,  más  tarde,  se  han  visto  confrontados  con  una  evolución  del \n'>
<LTTextBoxHorizontal(7) 92.700,615.019,522.460,626.309 'fenómeno  migratorio  hacia  un  asentamiento  estable  de  es

In [37]:
import pdfplumber

def detect_lines_in_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            lines = page.lines  # Extrae todas las líneas de la página

            # Filtrar líneas horizontales (y0 ≈ y1 indica una línea horizontal)
            horizontal_lines = [line for line in lines if abs(line["y0"] - line["y1"]) < 1]

            if horizontal_lines:
                print(f"Página {i+1}: {len(horizontal_lines)} ({(page.width, page.height)}) línea(s) horizontal(es) detectada(s).")
                for line in horizontal_lines:
                    print(f"  - Coordenadas: {line}")

detect_lines_in_pdf(articles[2])


In [30]:
import pdfquery
pdf = pdfquery.PDFQuery(articles[0])
pdf.load()
lines = pdf.pq('LTLine')
print(len(lines))
for line in lines:
    print(line.get('bbox'))

31
[92.7, 744.18, 519.36, 744.18]
[191.7, 317.16, 420.3, 317.16]
[191.7, 91.08, 420.3, 91.08]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]
[92.7, 744.18, 519.36, 744.18]


# Preprocesado

In [19]:
articles = [os.path.join("data", art) for art in os.listdir("data")]
len(articles)

34

In [ ]:
def extract_pages(articles):
  return [extract_pages(article) for article in articles]

def bound_delete(articles):
  pass

In [ ]:
import re

def limpiar_fechas(texto):
    # Regex mejorada para eliminar fechas en líneas individuales y en la misma línea
    regex = r""
    # Reemplazar coincidencias con una cadena vacía
    return re.sub(regex, "", texto).strip()

# Ejemplo de texto
texto = """
Recibido el 16 de enero de 2014
Aceptado el 6 de abril de 2014

Recibido: 25/09/2023. Aceptado: 06/12/2023. Publicado: 22/10/2024.

Recibido, 27 de noviembre de 2019
Segunda versión, 4 de marzo de 2020
Aceptado, 23 de marzo de 2020
"""

# Limpiar el texto
texto_limpio = limpiar_fechas(texto)

print(texto_limpio)
